In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [2]:
device = torch.device('cuda:0')

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [4]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [5]:
train_file = 'bernie_interviews/data/train.txt'
val_file = 'bernie_interviews/data/val.txt'
test_file = 'bernie_interviews/data/test.txt'

In [6]:
def read_file(filename):
    paths = []
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            paths.append('bernie_interviews/images/{}/{:04d}.jpg'.format(video, int(image)))
            labels.append(int(label))
            
    return paths, labels

In [7]:
train_paths, Y_train = read_file(train_file)
val_paths, Y_val = read_file(val_file)
test_paths, Y_test = read_file(test_file)

In [8]:
image_datasets = {
    'train': ImageDataset(train_paths, Y_train, transform = data_transforms['train']),
    'val_train': ImageDataset(val_paths, Y_val, transform = data_transforms['train']),
    'val': ImageDataset(val_paths, Y_val, transform = data_transforms['val']),
    'test': ImageDataset(test_paths, Y_test, transform = data_transforms['val'])
}

In [9]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=4,
                                              shuffle='train' in x,
                                              num_workers=4,
                                              pin_memory=True)
                                              for x in image_datasets}

In [10]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [11]:
dataset_sizes

{'train': 6835, 'val_train': 3026, 'val': 3026, 'test': 3563}

In [12]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [13]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

# Traditional Supervision

In [14]:
path = 'models/transfer_learning_tutorial'
for seed in range(2, 5):
    torch.manual_seed(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

    criterion = nn.BCEWithLogitsLoss().to(device)

    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------
train Loss: 0.1083 Acc: 0.9699 Pre: 0.8085 Rec: 0.2879 F1: 0.4246
TP: 76.0 TN: 6553.0 FP: 18.0 FN: 188.0
val Loss: 0.0506 Acc: 0.9874 Pre: 0.7922 Rec: 0.7349 F1: 0.7625
TP: 61.0 TN: 2927.0 FP: 16.0 FN: 22.0
test Loss: 0.0455 Acc: 0.9846 Pre: 0.7600 Rec: 0.7103 F1: 0.7343
TP: 76.0 TN: 3432.0 FP: 24.0 FN: 31.0

Epoch 1/24
----------
train Loss: 0.0744 Acc: 0.9823 Pre: 0.8950 Rec: 0.6136 F1: 0.7281
TP: 162.0 TN: 6552.0 FP: 19.0 FN: 102.0
val Loss: 0.1319 Acc: 0.9617 Pre: 0.4108 Rec: 0.9157 F1: 0.5672
TP: 76.0 TN: 2834.0 FP: 109.0 FN: 7.0
test Loss: 0.1249 Acc: 0.9635 Pre: 0.4502 Rec: 0.9720 F1: 0.6154
TP: 104.0 TN: 3329.0 FP: 127.0 FN: 3.0

Epoch 2/24
----------
train Loss: 0.0676 Acc: 0.9830 Pre: 0.8978 Rec: 0.6326 F1: 0.7422
TP: 167.0 TN: 6552.0 FP: 19.0 FN: 97.0
val Loss: 0.0564 Acc: 0.9861 Pre: 0.6881 Rec: 0.9036 F1: 0.7812
TP: 75.0 TN: 2909.0 FP: 34.0 FN: 8.0
test Loss: 0.0605 Acc: 0.9832 Pre: 0.6715 Rec: 0.8598 F1: 0.7541
TP: 92.0 TN: 3411.0 FP: 

test Loss: 0.0334 Acc: 0.9902 Pre: 0.7857 Rec: 0.9252 F1: 0.8498
TP: 99.0 TN: 3429.0 FP: 27.0 FN: 8.0

Training complete in 47m 18s
train val test
Epoch 0/24
----------
train Loss: 0.1098 Acc: 0.9710 Pre: 0.8438 Rec: 0.3068 F1: 0.4500
TP: 81.0 TN: 6556.0 FP: 15.0 FN: 183.0
val Loss: 0.0596 Acc: 0.9861 Pre: 0.6952 Rec: 0.8795 F1: 0.7766
TP: 73.0 TN: 2911.0 FP: 32.0 FN: 10.0
test Loss: 0.0476 Acc: 0.9882 Pre: 0.7519 Rec: 0.9065 F1: 0.8220
TP: 97.0 TN: 3424.0 FP: 32.0 FN: 10.0

Epoch 1/24
----------
train Loss: 0.0743 Acc: 0.9827 Pre: 0.9244 Rec: 0.6023 F1: 0.7294
TP: 159.0 TN: 6558.0 FP: 13.0 FN: 105.0
val Loss: 0.0479 Acc: 0.9898 Pre: 0.8333 Rec: 0.7831 F1: 0.8075
TP: 65.0 TN: 2930.0 FP: 13.0 FN: 18.0
test Loss: 0.0378 Acc: 0.9919 Pre: 0.8900 Rec: 0.8318 F1: 0.8599
TP: 89.0 TN: 3445.0 FP: 11.0 FN: 18.0

Epoch 2/24
----------
train Loss: 0.0746 Acc: 0.9804 Pre: 0.8652 Rec: 0.5833 F1: 0.6968
TP: 154.0 TN: 6547.0 FP: 24.0 FN: 110.0
val Loss: 0.0657 Acc: 0.9861 Pre: 0.7030 Rec: 0.8554 F1: 0

val Loss: 0.0446 Acc: 0.9911 Pre: 0.7917 Rec: 0.9157 F1: 0.8492
TP: 76.0 TN: 2923.0 FP: 20.0 FN: 7.0
test Loss: 0.0616 Acc: 0.9871 Pre: 0.8020 Rec: 0.7570 F1: 0.7788
TP: 81.0 TN: 3436.0 FP: 20.0 FN: 26.0

Training complete in 47m 9s
train val test
Epoch 0/24
----------
train Loss: 0.1062 Acc: 0.9719 Pre: 0.8462 Rec: 0.3333 F1: 0.4783
TP: 88.0 TN: 6555.0 FP: 16.0 FN: 176.0
val Loss: 0.1292 Acc: 0.9564 Pre: 0.3781 Rec: 0.9157 F1: 0.5352
TP: 76.0 TN: 2818.0 FP: 125.0 FN: 7.0
test Loss: 0.1379 Acc: 0.9548 Pre: 0.3937 Rec: 0.9346 F1: 0.5540
TP: 100.0 TN: 3302.0 FP: 154.0 FN: 7.0

Epoch 1/24
----------
train Loss: 0.0756 Acc: 0.9811 Pre: 0.8947 Rec: 0.5795 F1: 0.7034
TP: 153.0 TN: 6553.0 FP: 18.0 FN: 111.0
val Loss: 0.0613 Acc: 0.9851 Pre: 0.6727 Rec: 0.8916 F1: 0.7668
TP: 74.0 TN: 2907.0 FP: 36.0 FN: 9.0
test Loss: 0.0434 Acc: 0.9896 Pre: 0.7652 Rec: 0.9439 F1: 0.8452
TP: 101.0 TN: 3425.0 FP: 31.0 FN: 6.0

Epoch 2/24
----------
train Loss: 0.0688 Acc: 0.9813 Pre: 0.8579 Rec: 0.6174 F1: 0.71

train Loss: 0.0265 Acc: 0.9914 Pre: 0.9325 Rec: 0.8371 F1: 0.8822
TP: 221.0 TN: 6555.0 FP: 16.0 FN: 43.0
val Loss: 0.0468 Acc: 0.9891 Pre: 0.7451 Rec: 0.9157 F1: 0.8216
TP: 76.0 TN: 2917.0 FP: 26.0 FN: 7.0
test Loss: 0.0377 Acc: 0.9916 Pre: 0.8130 Rec: 0.9346 F1: 0.8696
TP: 100.0 TN: 3433.0 FP: 23.0 FN: 7.0

Training complete in 46m 41s


# Smooth Results

In [15]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [17]:
path = 'models/transfer_learning_tutorial'

log_file_img = open(os.path.join(
    path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(
    path, 'test_results_smoothed.log'), 'w')

for seed in range(5):
    print(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(
        os.path.join(path, 'seed_{}.pth'.format(seed))
    ))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

0
Acc: 0.9801 Pre: 0.6304 Rec: 0.8131 F1: 0.7102
TP: 87.0 TN: 3405.0 FP: 51.0 FN: 20.0
Smoothed stats:
(0.9912994667415099, 0.8877551020408163, 0.8130841121495327, 0.848780487804878, 87, 3445, 11, 20)
1
Acc: 0.9806 Pre: 0.6484 Rec: 0.7757 F1: 0.7064
TP: 83.0 TN: 3411.0 FP: 45.0 FN: 24.0
Smoothed stats:
(0.9918607914678642, 0.9431818181818182, 0.7757009345794392, 0.8512820512820513, 83, 3451, 5, 24)
2
Acc: 0.9902 Pre: 0.7857 Rec: 0.9252 F1: 0.8498
TP: 99.0 TN: 3429.0 FP: 27.0 FN: 8.0
Smoothed stats:
(0.9963513892786977, 0.9433962264150944, 0.9345794392523364, 0.9389671361502347, 100, 3450, 6, 7)
3
Acc: 0.9871 Pre: 0.8020 Rec: 0.7570 F1: 0.7788
TP: 81.0 TN: 3436.0 FP: 20.0 FN: 26.0
Smoothed stats:
(0.9915801291046871, 0.963855421686747, 0.7476635514018691, 0.8421052631578946, 80, 3453, 3, 27)
4
Acc: 0.9916 Pre: 0.8130 Rec: 0.9346 F1: 0.8696
TP: 100.0 TN: 3433.0 FP: 23.0 FN: 7.0
Smoothed stats:
(0.9969127140050519, 0.9528301886792453, 0.9439252336448598, 0.948356807511737, 101, 3451, 5, 6